In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
conn=engine.connect()

In [ ]:
Vacation_Start='2016-04-03'
Vacation_End='1026-04-13'

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)
# reflect the tables
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement=Base.classes.measurement
Station=Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
# I've made the decision to limit my search to the station in Honolulu so I'm not plotting multiple values on the same date.

# Calculate the date 1 year ago from the last data point in the database
YearStart=dt.datetime.date(dt.datetime.fromisoformat(engine.execute("SELECT max(date) FROM measurement WHERE station in (SELECT station FROM station WHERE name like '%HONOLULU%')").fetchall()[0][0]))- dt.timedelta(days=365)
print(f"The date the year of data will start on is {YearStart}.")

# Perform a query to retrieve the date and precipitation scores
# Since the start date is 2014-10-30, we know there isn't a leap day in our year of data.
engine.execute(f"SELECT date, prcp FROM measurement WHERE (date >= '{YearStart}') AND (station in (SELECT station FROM station WHERE name like '%HONOLULU%')) ORDER BY date LIMIT 10").fetchall()

# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.read_sql(f"SELECT date, prcp FROM measurement WHERE (date >= '{YearStart}') AND (station in (SELECT station FROM station WHERE name like '%HONOLULU%')) ORDER BY date", conn)
df.set_index('date', inplace=True)
# Sort the dataframe by date
df.head()

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
x_axis=df.index
bars=df.prcp
plt.figure(figsize=(10,5))
plt.bar(x_axis,bars,width=2)
plt.xticks([x for i,x in enumerate(x_axis) if i%30==0],rotation = 45)
plt.ylim(-.05,1.41)
plt.xlabel("Date")
plt.ylabel("Precipitation (in)")
plt.title("Precipitation by Date in Honolulu")
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
df.describe()

In [ ]:
# Design a query to show the number of stations available in this dataset.
Station_Count=engine.execute("SELECT COUNT(distinct station) FROM measurement").fetchall()[0][0]
print(f"The number of unique stations available in the dataset is {Station_Count}.")

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
engine.execute("SELECT station, COUNT(prcp) AS Records_Count, (SELECT name FROM station WHERE measurement.station = station.station) AS station_name FROM measurement GROUP BY station_name ORDER BY Records_Count DESC").fetchall()


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station.
Station_info=session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).filter(Measurement.station=='USC00519281').all()[0]
print(f"The temp. info for station USC00519281 is the following:\nLow: {Station_info[0]}\nHigh: {Station_info[1]}\nAvg: {round(Station_info[2],2)}")


In [ ]:
# Choose the station with the highest number of temperature observations.
engine.execute("SELECT station, COUNT(tobs) AS Records_Count, (SELECT name FROM station WHERE measurement.station = station.station) AS station_name FROM measurement GROUP BY station_name ORDER BY Records_Count DESC").fetchall()


In [ ]:
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
# Calculate the date 1 year ago from the last data point in the database
YearStart=dt.datetime.date(dt.datetime.fromisoformat(engine.execute("SELECT max(date) FROM measurement WHERE station = 'USC00519281'").fetchall()[0][0]))- dt.timedelta(days=365)
print(f"The date the year of data will start on is {YearStart}.")

# Save the query results as a Pandas DataFrame and set the index to the date column
Temp_df = pd.read_sql(f"SELECT date, tobs as temp FROM measurement WHERE (date >= '{YearStart}') AND (station = 'USC00519281') ORDER BY date", conn)
Temp_df.set_index('date', inplace=True)
# Sort the dataframe by date
Temp_df.head()

In [ ]:
#Plot the histogram
Temp_df["temp"].plot.hist(title="Temperature Distribution in Waihee", color="red",bins=12)
plt.xlabel("Temp (F)")
plt.show()

## Bonus Challenge

## Hawaii is reputed to enjoy mild weather all year. Is there a meaningful difference between the temperature in June and December?

In [ ]:
#Determine the average temp in June and December across all stations and all years of the data set.
June_Avg=engine.execute("SELECT avg(tobs), count(tobs), (SELECT SUM((tobs-(SELECT AVG(tobs) FROM measurement))*(tobs-(SELECT AVG(tobs) FROM measurement)) ) / (COUNT(tobs)-1)) AS Variance FROM measurement WHERE date LIKE '%-06-%' ORDER BY date").fetchall()[0]
Dec_Avg=engine.execute("SELECT avg(tobs), count(tobs), (SELECT SUM((tobs-(SELECT AVG(tobs) FROM measurement))*(tobs-(SELECT AVG(tobs) FROM measurement)) ) / (COUNT(tobs)-1)) AS Variance FROM measurement WHERE date LIKE '%-12-%' ORDER BY date").fetchall()[0]
print('(average          , count, variance        )')
print(June_Avg)
print(Dec_Avg)

In [ ]:
#Use the t-test to determine whether the difference in the means, if any, is statistically significant. Will you use a paired t-test, or an unpaired t-test? Why?



In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
